<a href="https://colab.research.google.com/github/sherry975/CSC-791-Project/blob/master/sentence_masking.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# !ls "/content/drive/My Drive/"

import os
os.chdir("/content/drive/My Drive/CSC791-NLP/791 NLP project/791_project_repo")


# ------------------------------------------------------------------
from sklearn.pipeline import Pipeline
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import FeatureUnion
import csv
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics.pairwise import linear_kernel
from sklearn.decomposition import TruncatedSVD
from sklearn.feature_extraction import DictVectorizer
from sklearn.metrics import roc_auc_score
import sys
import traceback
import string 

import re
import datetime
import pickle
import nltk
from nltk.corpus import stopwords
from nltk.stem import LancasterStemmer, WordNetLemmatizer
nltk.download('punkt')

from sklearn import preprocessing
from numpy import dot
from numpy.linalg import norm

def print_time(*args):
    text = ""
    for arg in args:
        text += arg
    print("[",datetime.datetime.now(),"]", text)


def curtime():    
    currentDT = datetime.datetime.now()
    return currentDT.strftime("%Y-%m-%d_%H%M%S")


def pickle_save(filename, data2pkl):
    fileObject = open(filename,'wb') 
    pickle.dump(data2pkl,fileObject)
    fileObject.close()

        
def pickle_load(filename):
    fileObject = open(filename,'rb') 
    data_unpkl = pickle.load(fileObject)
    fileObject.close()
    return data_unpkl

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
# process raw function words list and generate a txt
def get_function_words_list(filename):
    func_words_lst = []
    with open(filename, 'r', encoding='utf-8', newline='') as f:
        content = f.readlines()
        for row in content:
            if row[0] != "#":
                word = row.split(" ")[0]
                func_words_lst.append(word)
    return func_words_lst            

func_words_lst = get_function_words_list("raw_data/james_o_shea_277.txt")
func_words_set = set(func_words_lst)

# get puntuations as a set
punctuation_set = set([p for p in string.punctuation])



In [ ]:
# expand English contractions
# credit: https://gist.github.com/nealrs/96342d8231b75cf4bb82
cList = {
  "ain't": "am not",
  "aren't": "are not",
  "can't": "cannot",
  "can't've": "cannot have",
  "'cause": "because",
  "could've": "could have",
  "couldn't": "could not",
  "couldn't've": "could not have",
  "didn't": "did not",
  "doesn't": "does not",
  "don't": "do not",
  "hadn't": "had not",
  "hadn't've": "had not have",
  "hasn't": "has not",
  "haven't": "have not",
  "he'd": "he would",
  "he'd've": "he would have",
  "he'll": "he will",
  "he'll've": "he will have",
  "he's": "he is",
  "how'd": "how did",
  "how'd'y": "how do you",
  "how'll": "how will",
  "how's": "how is",
  "I'd": "I would",
  "I'd've": "I would have",
  "I'll": "I will",
  "I'll've": "I will have",
  "I'm": "I am",
  "I've": "I have",
  "isn't": "is not",
  "it'd": "it had",
  "it'd've": "it would have",
  "it'll": "it will",
  "it'll've": "it will have",
  "it's": "it is",
  "let's": "let us",
  "ma'am": "madam",
  "mayn't": "may not",
  "might've": "might have",
  "mightn't": "might not",
  "mightn't've": "might not have",
  "must've": "must have",
  "mustn't": "must not",
  "mustn't've": "must not have",
  "needn't": "need not",
  "needn't've": "need not have",
  "o'clock": "of the clock",
  "oughtn't": "ought not",
  "oughtn't've": "ought not have",
  "shan't": "shall not",
  "sha'n't": "shall not",
  "shan't've": "shall not have",
  "she'd": "she would",
  "she'd've": "she would have",
  "she'll": "she will",
  "she'll've": "she will have",
  "she's": "she is",
  "should've": "should have",
  "shouldn't": "should not",
  "shouldn't've": "should not have",
  "so've": "so have",
  "so's": "so is",
  "that'd": "that would",
  "that'd've": "that would have",
  "that's": "that is",
  "there'd": "there had",
  "there'd've": "there would have",
  "there's": "there is",
  "they'd": "they would",
  "they'd've": "they would have",
  "they'll": "they will",
  "they'll've": "they will have",
  "they're": "they are",
  "they've": "they have",
  "to've": "to have",
  "wasn't": "was not",
  "we'd": "we had",
  "we'd've": "we would have",
  "we'll": "we will",
  "we'll've": "we will have",
  "we're": "we are",
  "we've": "we have",
  "weren't": "were not",
  "what'll": "what will",
  "what'll've": "what will have",
  "what're": "what are",
  "what's": "what is",
  "what've": "what have",
  "when's": "when is",
  "when've": "when have",
  "where'd": "where did",
  "where's": "where is",
  "where've": "where have",
  "who'll": "who will",
  "who'll've": "who will have",
  "who's": "who is",
  "who've": "who have",
  "why's": "why is",
  "why've": "why have",
  "will've": "will have",
  "won't": "will not",
  "won't've": "will not have",
  "would've": "would have",
  "wouldn't": "would not",
  "wouldn't've": "would not have",
  "y'all": "you all",
  "y'alls": "you alls",
  "y'all'd": "you all would",
  "y'all'd've": "you all would have",
  "y'all're": "you all are",
  "y'all've": "you all have",
  "you'd": "you had",
  "you'd've": "you would have",
  "you'll": "you you will",
  "you'll've": "you you will have",
  "you're": "you are",
  "you've": "you have"
}

c_re = re.compile('(%s)' % '|'.join(cList.keys()))

def expandContractions(text, c_re=c_re):
    def replace(match):
        return cList[match.group(0)]
    return c_re.sub(replace, text)


In [ ]:
# mark content words 
def mark_sentences(in_filename, out_filename):
    csv_reader = csv.reader(open(in_filename, newline='', encoding='utf-8'), delimiter=',',
                            quotechar='"')

    row_cnt = 0
    marked_sentences = []

    REMOVE_PUNCTUATION = False
    USE_EXPANSION = True
    
    try:
        for row in csv_reader:
            # if row_cnt > 10:
            #     break
            if row[0] != "sentence_id":
                text = row[1]
                text_expanded = expandContractions(text)
                if USE_EXPANSION:
                    text = text_expanded
                function_set = func_words_set
                if REMOVE_PUNCTUATION:
                    text = text.translate(str.maketrans('', '', string.punctuation))
                else:
                    text = text.translate(str.maketrans({key: " {0} ".format(key) for key in punctuation_set}))
                    function_set = func_words_set|punctuation_set

                word_split = text.lower().split()  

                word_marked = [w if w in function_set else "#" for w in word_split]
                total_w_cnt = len(word_split)
                func_w_cnt = sum(map(word_split.count, function_set))
                mark_ratio = (total_w_cnt - func_w_cnt) / total_w_cnt
                marked_sentences.append(row + [word_split, word_marked, total_w_cnt, func_w_cnt, mark_ratio])
            row_cnt += 1
            
    except Exception as ex:
        sys.stderr.write('Exception\n')
        extype, exvalue, extrace = sys.exc_info()
        traceback.print_exception(extype, exvalue, extrace)

    # save a pickled list
    pickle_save(out_filename, marked_sentences)


    # with open(out_filename, 'w') as csvoutput:
    #     writer = csv.writer(csvoutput)
    #     writer.writerow(["sentence_id","text","len_in_words","len_in_char","formality","label","token", "marked_token", "num_total_token", "num_func_token","mark_ratio"])
    #     for item in marked_sentences:
    #         writer.writerow(item)

print_time("start to process.")

# mark_sentences("input_data/training.csv", 'output_data/training_masked_list.pkl')
mark_sentences("input_data/testing.csv", 'output_data/testing_masked_list.pkl')

# mark_sentences("input_data/training.csv", 'output_data/training_marked.csv')
# mark_sentences("input_data/testing.csv", 'output_data/testing_marked.csv')

print_time("done processing.")

[ 2019-11-19 20:53:45.147152 ] start to process.
[ 2019-11-19 20:53:45.553900 ] done processing.
